In [1]:
import sys

In [2]:
sys.getrefcount("qjwehfkjeh")

4294967295

In [3]:
sys.getrefcount("qjwehfkjeh!!")

3

In [4]:
sys.getrefcount(5)

4294967295

In [5]:
2 ** 32

4294967296

In [6]:
sys.getrefcount(256), sys.getrefcount(257)

(4294967295, 3)

In [7]:
x = 257

In [9]:
sys.getrefcount(257), sys.getrefcount(x)

(3, 2)

In [10]:
lst = [1, 2, 3]

In [11]:
sys.getrefcount(lst)

2

In [12]:
sys.getrefcount?

Signature: sys.getrefcount(object, /)
Docstring:
Return the reference count of object.

The count returned is generally one higher than you might expect,
because it includes the (temporary) reference as an argument to
getrefcount().
Type:      builtin_function_or_method

In [13]:
# object = lst

In [15]:
lst2 = lst

In [16]:
sys.getrefcount(lst), sys.getrefcount(lst2)

(3, 3)

In [17]:
lst3 = lst2

In [18]:
sys.getrefcount(lst), sys.getrefcount(lst2), sys.getrefcount(lst3)

(4, 4, 4)

In [19]:
del lst

In [21]:
sys.getrefcount(lst2), sys.getrefcount(lst3)

(3, 3)

In [22]:
lst2 = "replace"

In [23]:
sys.getrefcount(lst2), sys.getrefcount(lst3)

(4294967295, 2)

In [24]:
lst2 = "replace!"

In [25]:
sys.getrefcount(lst2)

2

In [28]:
rnd_str1 = "w" * 5
rnd_str2 = "s" * 5

In [29]:
rnd_str_total = rnd_str1 + rnd_str2

In [30]:
sys.getrefcount(rnd_str1), sys.getrefcount(rnd_str2), sys.getrefcount(rnd_str_total)

(4294967295, 4294967295, 5)

In [32]:
lst = [1, 2, 3]

def fn(obj):
    print(f"FN: {sys.getrefcount(obj)=}")

print(f"1: {sys.getrefcount(lst)=}")

fn(lst)

print(f"2: {sys.getrefcount(lst)=}")

1: sys.getrefcount(lst)=2
FN: sys.getrefcount(obj)=3
2: sys.getrefcount(lst)=2


In [33]:
class Animal:
    def __init__(self, name):
        self.name = name

    def __del__(self):
        print("DEL:", self.name, id(self))

In [34]:
tiger = Animal("tiger")

In [35]:
tg = tiger

In [36]:
sys.getrefcount(tg), sys.getrefcount(tiger)

(7, 7)

In [37]:
del tg

In [38]:
sys.getrefcount(tiger)

6

In [39]:
del tiger

In [40]:
tiger = Animal("tiger")
tg = tiger

print(f"{sys.getrefcount(tg)=}, {sys.getrefcount(tiger)=}")

del tg

print(f"{sys.getrefcount(tiger)=}")

del tiger

sys.getrefcount(tg)=3, sys.getrefcount(tiger)=3
sys.getrefcount(tiger)=2
DEL: tiger 4358293216


In [43]:
class Animal:
    def __init__(self, name, food):
        self.name = name
        self.food = food

    def __del__(self):
        print("DEL:", self.name, id(self))

In [48]:
tiger = Animal("tiger", None)
baboon = Animal("baboon", tiger)
tiger.food = baboon

print(f"{sys.getrefcount(tiger)=}, {sys.getrefcount(baboon)=}")

del tiger

print(f"{sys.getrefcount(baboon)=}")

del baboon

sys.getrefcount(tiger)=3, sys.getrefcount(baboon)=3
sys.getrefcount(baboon)=3


In [53]:
lst = [1, 2, 3]
lst.append(lst)
print(lst, id())

[1, 2, 3, [...]]


In [50]:
del lst

In [51]:
import ctypes

In [52]:
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]

In [54]:
PyObject.from_address(id(lst))

In [60]:
lst = [1, 2, 3]
lst.append(lst)
id_lst = id(lst)

print(PyObject.from_address(id_lst).refcnt)

del lst

print(PyObject.from_address(id_lst).refcnt)

2
1


In [66]:
tiger = Animal("tiger", None)
baboon = Animal("baboon", tiger)
tiger.food = baboon

id_tiger = id(tiger)
id_baboon = id(baboon)

print(f"{sys.getrefcount(tiger)=}, {sys.getrefcount(baboon)=}", id_tiger, id_baboon)
print("before:", PyObject.from_address(id_tiger).refcnt, PyObject.from_address(id_baboon).refcnt)

del tiger
del baboon

print("after:", PyObject.from_address(id_tiger).refcnt, PyObject.from_address(id_baboon).refcnt)

sys.getrefcount(tiger)=3, sys.getrefcount(baboon)=3 4369414048 4369401088
before: 2 2
after: 1 1


In [64]:
import gc

In [67]:
gc.collect()

DEL: tiger 4369414048
DEL: baboon 4369401088


1000

In [68]:
print("after gc:", PyObject.from_address(id_tiger).refcnt, PyObject.from_address(id_baboon).refcnt)

after gc: 0 0


In [70]:
for i in range(2):
    fish = Animal("fish", None)

DEL: fish 4357111152
DEL: fish 4369407904


In [71]:
print("after gc:", PyObject.from_address(id_tiger).refcnt, PyObject.from_address(id_baboon).refcnt)

after gc: 4363860580 4351096932


# Homework

In [72]:
def parse_args():
    args = Arguments()
    return args


def create_client(domain, num_threads):
    threads = [
        Client(domain=domain)
        for th in range(num_threads)
    ]
    for th in threads:
        th.start()
    for th in threads:
        th.join()


def run_client():
    args = parse_args()
    create_client(args.domain, args.num_threads)

    
    

In [76]:
import weakref
from functools import partial

In [81]:
class Car:
    def __init__(self, name):
        self.name = name

    def __del__(self):
        print("DEL:", self.name, id(self))


def final_dest(weak):
    obj = weak()
    print(f"Final dest {obj=}, {id(obj)=}")


honda = Car("honda")
weakref.finalize(honda, partial(final_dest, weakref.ref(honda)))

del honda

DEL: honda 4369307632
Final dest obj=None, id(obj)=4321898592


In [85]:
bmw = Car("bmw")
print("1:", sys.getrefcount(bmw))

wr_bmw = weakref.ref(bmw)

print("2:", sys.getrefcount(bmw))

bmw2 = wr_bmw()
print("3:", sys.getrefcount(bmw))

print(bmw is bmw2)

del bmw2

print("4:", sys.getrefcount(bmw))

del bmw

print("----")

bmw3 = wr_bmw()
print("bmw3", bmw3)

1: 2
2: 2
3: 3
True
4: 2
DEL: bmw 4369306144
----
bmw3 None


In [89]:
class Animal:
    def __init__(self, name):
        self.name = name
        self._food = None

    @property
    def food(self):
        return self._food

    @food.setter
    def food(self, food):
        self._food = weakref.ref(food)

    def __del__(self):
        print("DEL:", self.name, id(self))


tiger = Animal("tiger")
baboon = Animal("baboon")

tiger.food = baboon
baboon.food = tiger

id_tiger = id(tiger)
id_baboon = id(baboon)

print(f"{sys.getrefcount(tiger)=}, {sys.getrefcount(baboon)=}", id_tiger, id_baboon)
print("before:", PyObject.from_address(id_tiger).refcnt, PyObject.from_address(id_baboon).refcnt)

del tiger
del baboon

print("after:", PyObject.from_address(id_tiger).refcnt, PyObject.from_address(id_baboon).refcnt)

sys.getrefcount(tiger)=2, sys.getrefcount(baboon)=2 4374599232 4374598080
before: 1 1
DEL: tiger 4374599232
DEL: baboon 4374598080
after: 10 0


In [100]:
class Animal:

    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age


class AnimalSlots:
    __slots__ = ("name", "age")

    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age

In [101]:
tiger = Animal("tiger!", 1024)
lion = AnimalSlots("lion!", 2034)

In [92]:
print(tiger.name, tiger.age, lion.name, lion.age)

tiger! 1024 lion! 2034


In [93]:
tiger.name, tiger.age, lion.name, lion.age = "tiger_1!!", 4455, "lion123)^", 9988

In [94]:
print(tiger.name, tiger.age, lion.name, lion.age)

tiger_1!! 4455 lion123)^ 9988


In [95]:
tiger.food = "meat"

In [96]:
print(tiger.name, tiger.age, tiger.food, lion.name, lion.age)

tiger_1!! 4455 meat lion123)^ 9988


In [97]:
lion.food = "meat"

AttributeError: 'AnimalSlots' object has no attribute 'food'

In [102]:
tiger.__dict__

{'name': 'tiger!', 'age': 1024}

In [103]:
lion.__dict__

AttributeError: 'AnimalSlots' object has no attribute '__dict__'

In [104]:
Animal.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Animal.__init__(self, name: str, age: int)>,
              '__dict__': <attribute '__dict__' of 'Animal' objects>,
              '__weakref__': <attribute '__weakref__' of 'Animal' objects>,
              '__doc__': None})

In [105]:
AnimalSlots.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name', 'age'),
              '__init__': <function __main__.AnimalSlots.__init__(self, name: str, age: int)>,
              'age': <member 'age' of 'AnimalSlots' objects>,
              'name': <member 'name' of 'AnimalSlots' objects>,
              '__doc__': None})

In [106]:
sys.getsizeof(Animal), sys.getsizeof(AnimalSlots)

(1704, 936)

In [108]:
sys.getsizeof(tiger), sys.getsizeof(lion)

(48, 48)

In [109]:
tiger.xx = "xx"
tiger.yy = "yy"
tiger.zz = "zz"
tiger.zz1 = "zz"
tiger.zz2 = "zz"
tiger.zz3 = "zz"

In [110]:
sys.getsizeof(tiger), sys.getsizeof(lion)

(48, 48)

In [111]:
sys.getsizeof(tiger.__dict__)

296

In [113]:
sys.getsizeof({}), sys.getsizeof({"123": "345"})

(64, 184)

In [114]:
dct = {}
for i in range(8):
    before = sys.getsizeof(dct)
    dct[f"xxxx_{i}"] = f"{i}_val"
    after = sys.getsizeof(dct)

    print(f"{len(dct)=}, {before=}, {after=}")

len(dct)=1, before=64, after=184
len(dct)=2, before=184, after=184
len(dct)=3, before=184, after=184
len(dct)=4, before=184, after=184
len(dct)=5, before=184, after=184
len(dct)=6, before=184, after=272
len(dct)=7, before=272, after=272
len(dct)=8, before=272, after=272
